In [5]:
import pandas as pd

# Load the templates (if needed for reference)
final_templates = pd.read_csv(r'C:\Users\lathe\Desktop\rag model training\dataset\final_templates.csv')
print("Final Templates (first 5 rows):")
print(final_templates.head())

# Load the data for each level:
df_level1 = pd.read_csv(r'C:\Users\lathe\Desktop\rag model training\dataset\knowledge_plan_grouping.csv')
df_level2 = pd.read_csv(r'C:\Users\lathe\Desktop\rag model training\dataset\knowledge_premium_grouping.csv')
df_level3 = pd.read_csv(r'C:\Users\lathe\Desktop\rag model training\dataset\knowlede_plan_disease_combinations.csv')
df_level4 = pd.read_csv(r'C:\Users\lathe\Desktop\rag model training\dataset\insurance_plans_by_disease.csv')  # used for both level 4 and level 5

print("\nData Loaded:")
print("Level 1 data shape:", df_level1.shape)
print("Level 2 data shape:", df_level2.shape)
print("Level 3 data shape:", df_level3.shape)
print("Level 4 data shape:", df_level4.shape)


Final Templates (first 5 rows):
  hierarchy_level                                         user_query  \
0          Level1  For the plan '{Plan Name}' and explain its pre...   
1          Level1  For the plan '{Plan Name}' and what premium op...   
2          Level1  For the plan '{Plan Name}' and what is its pre...   
3          Level1  Regarding plan '{Plan Name}' and what premium ...   
4          Level1  I need details for plan '{Plan Name}' and what...   

  target_columns                               output_template  
0   Premium Type  It is offered in the {Premium Type} category  
1   Premium Type  The available premium type is {Premium Type}  
2   Premium Type  It is offered in the {Premium Type} category  
3   Premium Type        Its premium category is {Premium Type}  
4   Premium Type            The premium type is {Premium Type}  

Data Loaded:
Level 1 data shape: (7, 7)
Level 2 data shape: (3, 9)
Level 3 data shape: (21, 8)
Level 4 data shape: (900, 13)


In [4]:
import re

def determine_level(prompt):
    """Determine the hierarchy level based on keywords in the prompt."""
    prompt_lower = prompt.lower()
    if "amount" in prompt_lower:
        return 5
    elif "disease" in prompt_lower:
        # If both plan and premium are mentioned, assume Level 4
        if "plan" in prompt_lower and "premium" in prompt_lower:
            return 4
        else:
            return 4  # default if disease is mentioned
    elif "plan" in prompt_lower and "premium" in prompt_lower:
        return 3
    elif "premium" in prompt_lower:
        return 2
    elif "plan" in prompt_lower:
        return 1
    else:
        return 1  # default

def extract_inputs(prompt):
    """
    Extract input values from the prompt.
    We assume the prompt contains markers like:
      Plan: 'Individual Health Insurance'
      Premium: 'Basic'
      Disease: 'Heart Attack'
      Amount: 500000
    """
    inputs = {}
    plan_match = re.search(r"Plan(?:\s*Name)?:\s*['\"]([^'\"]+)['\"]", prompt, re.IGNORECASE)
    if plan_match:
        inputs["Plan Name"] = plan_match.group(1).strip()
    premium_match = re.search(r"Premium(?:\s*Type)?:\s*['\"]([^'\"]+)['\"]", prompt, re.IGNORECASE)
    if premium_match:
        inputs["Premium Type"] = premium_match.group(1).strip()
    disease_match = re.search(r"Disease(?:\s*Name)?:\s*['\"]([^'\"]+)['\"]", prompt, re.IGNORECASE)
    if disease_match:
        inputs["Disease"] = disease_match.group(1).strip()
    amount_match = re.search(r"Amount:\s*([\d]+)", prompt, re.IGNORECASE)
    if amount_match:
        inputs["Amount"] = float(amount_match.group(1))
    return inputs

# Test the helper functions:
example_prompt = "Plan: 'Individual Health Insurance', Premium: 'Basic'"
print("\nDetermined level for example prompt:", determine_level(example_prompt))
print("Extracted inputs:", extract_inputs(example_prompt))



Determined level for example prompt: 3
Extracted inputs: {'Plan Name': 'Individual Health Insurance', 'Premium Type': 'Basic'}


In [5]:
def retrieve_record(prompt):
    """Given a prompt, determine level and extract matching record from the corresponding dataframe."""
    level = determine_level(prompt)
    inputs = extract_inputs(prompt)
    
    if level == 1:
        df = df_level1.copy()
        if "Plan Name" in inputs:
            df = df[df["Plan Name"].str.contains(inputs["Plan Name"], case=False, na=False)]
    elif level == 2:
        df = df_level2.copy()
        if "Premium Type" in inputs:
            df = df[df["Premium Type"].str.contains(inputs["Premium Type"], case=False, na=False)]
    elif level == 3:
        df = df_level3.copy()
        if "Plan Name" in inputs:
            df = df[df["Plan Name"].str.contains(inputs["Plan Name"], case=False, na=False)]
        if "Premium Type" in inputs:
            df = df[df["Premium Type"].str.contains(inputs["Premium Type"], case=False, na=False)]
    elif level in [4, 5]:
        df = df_level4.copy()
        if "Plan Name" in inputs:
            df = df[df["Plan Name"].str.contains(inputs["Plan Name"], case=False, na=False)]
        if "Premium Type" in inputs:
            df = df[df["Premium Type"].str.contains(inputs["Premium Type"], case=False, na=False)]
        if "Disease" in inputs:
            df = df[df["Disease"].str.contains(inputs["Disease"], case=False, na=False)]
    return level, inputs, df

def compute_out_of_pocket(record, amount, premium_type):
    """
    Compute out-of-pocket expense based on amount, premium type, and record values.
    Uses:
      Deductible %: Basic=5%, Lite=10%, Premier=15%
      Copay %: Basic=20%, Lite=10%, Premier=5%
    """
    premium_type = premium_type.lower()
    if "basic" in premium_type:
        deductible_pct = 5
        copay_pct = 20
    elif "lite" in premium_type:
        deductible_pct = 10
        copay_pct = 10
    elif "premier" in premium_type:
        deductible_pct = 15
        copay_pct = 5
    else:
        deductible_pct = 5
        copay_pct = 20

    # Get maximum coverage from record (if available)
    try:
        max_coverage = float(record.get("Maximum Coverage", 0))
    except:
        max_coverage = 0

    if max_coverage > 0 and amount > max_coverage:
        return f"Claim amount {amount} exceeds maximum coverage of {max_coverage}."
    
    deductible_value = amount * deductible_pct / 100
    updated_amount = amount - deductible_value
    copay_value = updated_amount * copay_pct / 100
    final_out_of_pocket = deductible_value + copay_value
    return final_out_of_pocket

def retrieve_answer(prompt):
    """Given a prompt, retrieve the matching record and return an answer string."""
    level, inputs, df = retrieve_record(prompt)
    if df.empty:
        return "No matching record found."
    record = df.iloc[0].to_dict()  # take the first match
    
    if level == 5:
        if "Amount" in inputs:
            result = compute_out_of_pocket(record, inputs["Amount"], inputs.get("Premium Type", "basic"))
            answer = (f"For the plan '{record.get('Plan Name', 'Unknown')}' with premium '{record.get('Premium Type', 'Unknown')}' "
                      f"and disease '{record.get('Disease', 'Unknown')}', the calculated out-of-pocket expense is {result}.")
        else:
            try:
                default_amount = float(record.get("Maximum Coverage", 0))
            except:
                default_amount = 0
            result = compute_out_of_pocket(record, default_amount, inputs.get("Premium Type", "basic"))
            answer = (f"For the plan '{record.get('Plan Name', 'Unknown')}' with premium '{record.get('Premium Type', 'Unknown')}' "
                      f"and disease '{record.get('Disease', 'Unknown')}', assuming maximum coverage as amount, "
                      f"the calculated out-of-pocket expense is {result}.")
    else:
        # For levels 1-4, simply return the target columns from the record.
        # (You can format this more nicely if desired.)
        answer = "Retrieved data: " + ", ".join([f"{k}: {v}" for k, v in record.items()])
    return answer

# Test the retriever with an example prompt:
example_prompt = "Plan: 'Individual Health Insurance', Premium: 'Basic', Disease: 'Heart Attack', Amount: 500000"
print("\nRetrieval Example:")
print(retrieve_answer(example_prompt))



Retrieval Example:
For the plan 'Individual Health Insurance' with premium 'Basic' and disease 'Heart Attack', the calculated out-of-pocket expense is Claim amount 500000.0 exceeds maximum coverage of 420379.0..


In [6]:
def data_retriever_model(user_prompt):
    """
    A simple data retriever model that:
      1. Determines the hierarchy level from the user prompt.
      2. Extracts input values.
      3. Retrieves the corresponding record(s) from the appropriate dataset.
      4. If level 5 and an amount is provided, computes the final out-of-pocket calculation.
      5. Returns an answer string.
    """
    answer = retrieve_answer(user_prompt)
    return answer

# Test the model with several example prompts:
prompts = [
    "Plan: 'Family Health Insurance'",  # Level 1
    "Premium: 'Lite'",  # Level 2
    "Plan: 'Individual Health Insurance', Premium: 'Basic'",  # Level 3
    "Plan: 'Individual Health Insurance', Premium: 'Basic', Disease: 'Heart Attack'",  # Level 4
    "Plan: 'Individual Health Insurance', Premium: 'Basic', Disease: 'Heart Attack', Amount: 300000"  # Level 5
]

for p in prompts:
    print("\nUser prompt:", p)
    print("Answer:", data_retriever_model(p))



User prompt: Plan: 'Family Health Insurance'
Answer: Retrieved data: Plan Name: Family Health Insurance, Premium Type: Basic, Lite, Premier, DisCount: 142, Diseases: Accidental Injury, Acid Reflux, Acute Myocardial Infarction, Age-related Macular Degeneration, Alzheimer's Disease, Amputation, Angina Pectoris, Anxiety Disorders, Aorta Surgery, Aortic Dissection, Appendicitis, Arteriosclerosis, Arthritis, Asthma, Asymptomatic Covid, Atrial Fibrillation, Atrial Flutter, Attention Deficit Disorder, Binge Eating Disorder, Bipolar Disorder, Blunt Force Trauma, Brain Surgery, Bronchiolitis, Burns, COPD, Cancer, Cardiomyopathy, Chemical Burns, Chronic Back Pain, Chronic Fatigue Syndrome, Chronic Kidney Disease, Chronic Liver Disease, Chronic Lung Disease, Chronic Pancreatitis, Chronic Sinusitis, Compound Fracture, Concussion, Congestive Heart Failure, Contusions, Coronary Artery Disease, Coronary Heart Disease, Covid Cardiac Complications, Covid Complications, Covid Endothelial Dysfunction, C